In [ ]:
"""import requests
from bs4 import BeautifulSoup"""

In [ ]:
"""# 크롤링할 웹 페이지 URL
url = 'https://example.com'

# 웹 페이지에 GET 요청을 보냄
response = requests.get(url)

# 응답이 성공적인지 확인
if response.status_code == 200:
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 원하는 데이터를 찾기 위해 HTML 요소를 선택
    # 예를 들어, 웹 페이지의 제목을 가져오기 위해 <title> 태그를 선택
    title = soup.find('title').text
    
    # 결과 출력
    print("웹 페이지 제목:", title)
else:
    print("요청 실패:", response.status_code)"""

In [7]:
import csv

def read_csv_file(file_path):
    data = []
    with open(file_path, newline='', encoding='utf-8-sig') as csvfile:
        reader = csv.reader(csvfile)
        index = 0
        for row in reader:
            clean_row = [col.replace(" ",'') for col in row]
            clean_row = [index] + clean_row
            data.append(clean_row)
            index += 1
    return data

file_path = 'NVDA 과거 데이터.csv'

csv_data = read_csv_file(file_path)
print(csv_data)

[[0, '날짜', '종가', '시가', '고가', '저가', '거래량', '변동%'], [1, '2024-03-13', '908.88', '910.55', '915.04', '884.35', '62.73M', '-1.12%'], [2, '2024-03-12', '919.13', '880.49', '919.60', '861.50', '65.44M', '7.16%'], [3, '2024-03-11', '857.74', '864.29', '887.97', '841.66', '67.84M', '-2.00%'], [4, '2024-03-08', '875.28', '951.38', '974.00', '865.06', '114.23M', '-5.55%'], [5, '2024-03-07', '926.69', '901.58', '927.67', '896.02', '58.67M', '4.47%'], [6, '2024-03-06', '887.00', '880.22', '897.24', '870.30', '57.49M', '3.18%'], [7, '2024-03-05', '859.64', '852.70', '860.97', '834.17', '51.46M', '0.85%'], [8, '2024-03-04', '852.37', '841.30', '876.95', '837.19', '60.68M', '3.60%'], [9, '2024-03-01', '822.79', '800.00', '823.00', '794.35', '47.91M', '4.00%'], [10, '2024-02-29', '791.12', '790.94', '799.90', '783.50', '49.88M', '1.87%'], [11, '2024-02-28', '776.63', '776.20', '789.33', '771.25', '37.95M', '-1.32%'], [12, '2024-02-27', '787.01', '793.81', '794.80', '771.62', '38.94M', '-0.49%'], [13, 

In [ ]:
# import list
import os
import random

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
from torch import nn, tensor
from torch.utils.data import Dataset

In [ ]:
# utils

# 파일 이동하며 학습하지 않으므로 path 부분 수정 필요

def split_dataset(csv_path: os.PathLike, split_rate: float = 0.2) -> None:

	root_dir = os.path.dirname(csv_path)

	df = pd.read_csv(csv_path)
	size = len(df)
	indices = list(range(size))

	# 리스트 셔플
	random.shuffle(indices)

	# train test 나눌 포인트 설정
	split_point = int(split_rate * size)

	# train test 나누기
	test_ids = indices[:split_point]
	train_ids = indices[split_point:]

	# train test 정보 loc 이용하여 묶어서 csv 파일 생성
	test_df = df.loc[test_ids]
	test_df.to_csv(os.path.join(root_dir, 'test_answer.csv'), index=False)
	train_df = df.loc[train_ids]
	train_df.to_csv(os.path.join(root_dir, 'train_answer.csv'), index=False)

In [ ]:
# dataset

# 건들지도 않았음 수정 필요

class Cifar10Dataset(Dataset):
    def __init__(self, image_dir,label_path, transform):    # 변수 초기화
        super().__init__()

        self.image_dir = image_dir  # 디렉토리
        self.labels = pd.read_csv(label_path)   # csv 파일에서 레이블 읽기
        self.transform = transform  # ?

    def __len__(self):      # Cifar10Dataset 길이 반환
        return len(self.labels) # 레이블 길이 = 데이터셋 길이
    
    def __getitem__(self, index):   # Cifat10Dataset 아이템 반환
        image_id = self.labels.loc[index]   # 레이블에서 index에 맞는 정보 묶어서 가져오기
        image = Image.open(os.path.join(self.image_dir, f"{image_id['id']}.png")).convert('RGB') # RGB로 변환하여 이미지 파일 가져오기
        label = CLASSES.index(image_id['label'])    # 레이블에 맞는 클래스명 저장

        if self.transform is not None:
            image = self.transform(image)   # ?

        return image, label # 이미지와 레이블 반환

In [ ]:
# model

# 모델 다시 만들어야 함 이미지 아니라서 컨볼루션 힘들 듯

class Net(nn.Module):
    # 학습과 추론에 사용되는 간단한 뉴럴 네트워크

    def __init__(self, num_classes: int) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)
    
    def forward(self, x: Tensor) -> Tensor:
        # 피드 포워드 진행하는 함수
        
        """
        param x: 입력 이미지
        type x: Tensor
        return: 입력 이미지에 대한 예측값(클래스값)
        rtype: Tensor
        """

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [ ]:
# train

In [ ]:
# test